In [ ]:
using Revise
using SSMCMain, SSMCMain.ModifiedMiCRM, MLSolver

In [ ]:
using CairoMakie, ProgressLogging

## Basic no space solve

In [ ]:
D = fill(0., 2, 3, 3)
D[1,:,:] .= [0. 0. 0.; 0. 0. 0.; 1 0. 0.]
D[2,:,:] .= [0. 0. 0.; 1. 0. 0.; 0. 0. 0.]
p = make_mmicrm_smart(2, 3, 200;
    D=SArray{Tuple{2,3,3}}(D),
    K=[3., 0., 0.],
    r=[1., 1., 1.],
    l=[1. 0. 0.; 1. 0. 0.],
    c=[2. 2. 0.; 2. 0. 2.],
    m=[0.5, 0.9],
    u0=[1., 1., 0., 0., 0.],
    u0rand=0.
)
s = solve(p)
display(plot_mmicrm_sol(s))
# display(p.p.c)
# display(p.p.D[1,:,:])
# display(p.p.D[2,:,:])
#show(s.u[end])
# param_summary(p.p)

## Linear stability analysis

In [ ]:
symsol = symbolic_solve_ode_ml(p)

In [ ]:
sssol = solve(SteadyStateProblem(p), DynamicSS())
show(sssol)

In [ ]:
ks = LinRange(0., 10., 10000)
Ds = [0.01, 0.01, 1e2, 1., 1.]
usol = s.u[end]
#usol = sssol.u
#usol = Symbolics.value.(getindex.(symsol[2], 2))
show(uninplace(mmicrmfunc!)(sssol.u, p.p, 0))

In [ ]:
lambdas = do_linstab_for_ks(ks, p, Ds, usol);
@show maximum(real(lambdas))
faa = plot_linstab_lambdas(ks, lambdas[:,:])
ylims!(faa.axis, (-0.5, 0.5))
#xlims!(faa.axis, (0., 2.))
faa

In [ ]:
ff = linstab_make_full_func(p.p, usol, Ds)
kmax_i = argmax(maximum.(eachrow(real(lambdas))))
kmax = ks[kmax_i]
kmax_i, kmax

In [ ]:
kmax_es = ff(kmax);
kmax_es.vectors[:,1]

## Spatial stuff - 1D!

In [ ]:
using SSMCMain.SpaceMMiCRM

In [ ]:
ssize = (200,)
su0 = expand_u0_to_size(ssize, usol)
psu0 = perturb_u0_uniform(get_Ns(p.p)..., su0, 0.2, 0.2)
space = make_cartesianspace_smart(length(ssize); dx=0.2)
sp = make_smmicrm_problem_safe(psu0, 10000, p.p, SA[Ds...], space)

In [ ]:
sps = solve(sp)
display(sps.retcode)
display(sps.destats.nf)
plot_1dsmmicrm_sol_snap(sps, 50.0)